**Overview**

In this notebook, we applied the Elastic Net model on the proteomics data to predict the treatment groups (Placabo vs. GRF6021) at each time point V3, V4a, V4b and V5. Then we utilized k-means to cluster the proteins and applied the over-representation pathway analysis to annotate each cluster at V3 timepoint.

**1.Elastic Net Prediction at Each Timepoint (V3, V4a, V4b, V5)**

In [ ]:
#data processing
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt
from pandas import Series,DataFrame
import canopy
#EN model
from statannot import add_stat_annotation
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
#k-means and pathway analysis
from sklearn.manifold import TSNE
import networkx as nx
import plotly.graph_objects as go
from sklearn.cluster import KMeans
import colorcet as cc
import gseapy as gp

In [ ]:
#process the data for EN model

def process_time_point(df, time_point):
    # Filter the DataFrame for the given time point and remove the first column
    df_withid = df[df['Time point'] == time_point].iloc[:, 1:]
    df_withid.reset_index(inplace=True,drop=True)
    # Extract features and apply log2 transformation
    X_df = df_withid.iloc[:, 2:]
    X_df = np.log2(X_df)
    
    # Standardize the features
    scaler = StandardScaler()
    X_df_stan = pd.DataFrame(scaler.fit_transform(X_df), index=X_df.index, columns=X_df.columns)
    
    # Map the 'Treatment' column to numerical values
    df_withid['Treatment'] = df_withid['Treatment'].map({'GRF6021': 1, 'Placebo': 2})
    
    # Extract the target variable
    y_pro = df_withid.iloc[:, 1]
    
    return X_df_stan, y_pro

# Time points to process
time_points = ['V3', 'V4a', 'V4b', 'V5']

# Dictionaries to hold the processed features and targets
X_pro_stan = {}
y_pro = {}

# Process each time point
#read the data
all_pro=pd.read_excel("Proteomics_all.xlsx")
for tp in time_points:
    X_pro_stan[tp], y_pro[tp] = process_time_point(all_pro, tp)


In [ ]:
import re
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

def make_predictions(X,y):
    # X: assume X is a dataframe of training data
    # y: y is the label of the data
    # 
    # return a dataframe of the ans 
    #
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]
    rows = max(X.index) + 1
    ans_matrix = []
    auc_roc = []
    feature_importance={}
    #Iterate 100 experiments, in each experiment, we randomly select 50% of the data as training data and the rest 50% as testing data.
    for i in range(100):
        # a ans_list is a single column in the orignial ans matrix
        ans_list = [None] * rows
        df_train = X.sample(frac=0.5)
        df_test = X.drop(df_train.index)
        # initiate an empty model   
        # define model
        reg = LogisticRegression(penalty = 'elasticnet', solver = 'saga', l1_ratio = 0.9)
        # train the model        
        reg.fit(df_train, y[df_train.index])
        # use the trained model to make predictions
        df_test_label = reg.predict(df_test)
        importance=reg.coef_[0]
        # summarize feature importance
        
        for i,v in enumerate(importance):
            if i not in feature_importance:
                feature_importance[i]=[]
            feature_importance[i].append(v)
        # assign the predicted results to the ans_list
        start = 0
        for j in df_test.index:
            ans_list[j] = df_test_label[start]
            start += 1
            
        ans_matrix.append(ans_list)
    
    ans = np.array(ans_matrix)
    ans = np.transpose(ans)
    df_ans = pd.DataFrame(ans)
    
    return df_ans,feature_importance

In [ ]:
# Initialize dictionaries to store the predictions, feature importances, and result DataFrames
ans = {}
fi = {}
df_result = {}
df_withid = {}

# Loop through each time point, make predictions, and prepare the result DataFrame
time_points=['V3','V4a','V4b','V5']

for tp in time_points:
    
    # Making predictions
    ans[tp], fi[tp] = make_predictions(X_pro_stan[tp], y_pro[tp])
    ans[tp]['mean'] = ans[tp].mean(axis=1) 
    
    # Concatenating the 'Treatment' and 'mean' data
    df_withid[tp] = all_pro[all_pro['Time point'] == tp]
    df_withid[tp].reset_index(inplace=True,drop=True)
    df_result[tp] = pd.concat([df_withid[tp]['Treatment'], ans[tp]['mean']], axis=1)


In [ ]:
#visualize the prediction results

fig, axs = plt.subplots(2, 2,figsize=(20,20))
fig.suptitle('Elastic Net Model of Proteomics(7272) Data Prediction Results',fontsize=25)
sns.set(style="whitegrid")

#V3
sns.boxplot(ax=axs[0,0],x="Treatment",y="mean",data=df_result['V3'],order=["GRF6021","Placebo"])
axs[0, 0].set_title('V3',fontsize=32)
axs[0, 0].set_ylim(0.8,2.4)
axs[0, 0].set_xlabel('Treatment', fontsize=20)
axs[0, 0].tick_params(axis='both', labelsize=20)
axs[0, 0].set_ylabel('Predicted value', fontsize=20)
add_stat_annotation(ax=axs[0,0], data=df_result_V3, x="Treatment", y="mean",
                    box_pairs=[("GRF6021", "Placebo")],
                    test='t-test_ind', text_format='simple', loc='inside', verbose=2)

#V4a
sns.boxplot(ax=axs[0,1],x="Treatment",y="mean",data=df_result['V4a'],order=["GRF6021","Placebo"])
axs[0, 1].set_title('V4a',fontsize=32)
axs[0, 1].set_ylim(0.8,2.4)
axs[0, 1].set_xlabel('Treatment', fontsize=20)
axs[0, 1].tick_params(axis='both', labelsize=20)
axs[0, 1].set_ylabel('Predicted value', fontsize=20)
add_stat_annotation(ax=axs[0,1], data=df_result_V4a, x="Treatment", y="mean",
                    box_pairs=[("GRF6021", "Placebo")],
                    test='t-test_ind', text_format='simple', loc='inside', verbose=2)
#V4b
sns.boxplot(ax=axs[1,0],x="Treatment",y="mean",data=df_result_V4b,order=["GRF6021","Placebo"])
axs[1, 0].set_title('V4b',fontsize=32)
axs[1, 0].set_ylim(0.8,2.4)
axs[1, 0].set_xlabel('Treatment', fontsize=20)
axs[1, 0].tick_params(axis='both', labelsize=20)
axs[1, 0].set_ylabel('Predicted value', fontsize=20)
add_stat_annotation(ax=axs[1,0], data=df_result['V4b'], x="Treatment", y="mean",
                    box_pairs=[("GRF6021", "Placebo")],
                    test='t-test_ind', text_format='simple', loc='inside', verbose=2)
#V5
sns.boxplot(ax=axs[1,1],x="Treatment",y="mean",data=df_result['V5'],order=["GRF6021","Placebo"])
axs[1, 1].set_title('V5',fontsize=32)
axs[1, 1].set_ylim(0.8,2.4)
axs[1, 1].set_xlabel('Treatment', fontsize=20)
axs[1, 1].tick_params(axis='both', labelsize=20)
axs[1, 1].set_ylabel('Predicted value', fontsize=20)
add_stat_annotation(ax=axs[1,1], data=df_result_V5, x="Treatment", y="mean",
                    box_pairs=[("GRF6021", "Placebo")],
                    test='t-test_ind', text_format='simple', loc='inside', verbose=2)

for ax in axs.flat:
    ax.set(ylabel='Prediction value')
    
plt.savefig('Proteomics_remove1_boxplot_7272.png', dpi=300)

plt.show()

**2. K-means Clustering with Top Pathways**

In [ ]:
#Extract V3 data
df_V3=df_withid['V3']
df_V3_feature=df_V3.iloc[:,3:]
#target column
df_V3_target=df_V3.iloc[:,2]
df_V3_all=pd.concat([df_V3_feature,df_V3_target],axis=1)

In [ ]:
#Caculate the correlation matrix of all significant features
correlation_matrix = df_V3_feature.corr()
correlation_matrix

In [ ]:
#Apply Tsne to layout the nodes
x = df_V3_feature.T
tsne = TSNE(n_components=2, random_state=123)
z = tsne.fit_transform(x) 

In [ ]:
#Visualize the plot
df = pd.DataFrame()
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2",
                palette=sns.color_palette("hls", 3),
                data=df).set(title="V3 T-SNE projection")

In [ ]:
#set up the position of each node same as the Tsne
my_pos={}
for idx,i in enumerate(list(correlation_matrix.index)):
    my_pos[i]=tuple((z)[idx].tolist())
print(my_pos)
print(len(my_pos))

In [ ]:
#create a dataframe to store the position 
my_pos_df=pd.DataFrame.from_dict(my_pos).T
my_pos_df.columns=['X','Y']
my_pos_df

In [ ]:
#Utilize networkx to generate the plot
node_list=list(correlation_matrix.columns)
#node_list=short_name_list
# initialize nodes with node properties
G = nx.Graph()
G.add_nodes_from(my_pos.keys(),size=10)
for n, p in my_pos.items():
    G.nodes[n]['pos'] = p
nx.draw(G,pos=my_pos,node_color='lightgray',node_size=50,with_labels=True,font_size=1)


In [ ]:
node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']#record the position
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        reversescale=True,
        color='grey',
        size=10,
        line_width=[]))
G.nodes

In [ ]:
node_text = []
for node in G.nodes:
    node_text.append(node)

node_trace.text = node_text
fig = go.Figure(data=[node_trace],
             layout=go.Layout(
                title='<br>V3 Tsne',
                titlefont_size=16,
                showlegend=False,
                #hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=True, zeroline=False, showticklabels=True),
                yaxis=dict(showgrid=True, zeroline=False, showticklabels=True))
                )
fig.show()
fig.write_html("Interactive_tsne.html")

In [ ]:
#k-means
x=node_x
y=node_y
#use Elbow method to derermine the number of clusters
data = list(zip(x, y))
inertias = []

for i in range(1,30):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,30), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
#Decide to use 20 as the number of clusters
kmeans = KMeans(n_clusters=20,random_state=66)
kmeans.fit(z)
plt.scatter(x, y, c=kmeans.labels_)
plt.show()

In [ ]:
labels = kmeans.labels_

# Get cluster centroids
centroids = kmeans.cluster_centers_
plt.scatter(z[:, 0], z[:, 1], c=labels, cmap='viridis', alpha=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='X', s=200, color='red')

plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('K-means Clustering in t-SNE Space')
#plt.legend(*sc.legend_elements(), title='clusters')
plt.show()

In [ ]:
plt.scatter(z[:, 0], z[:, 1], c=labels, cmap='viridis', alpha=0.5)
#plt.scatter(centroids[:, 0], centroids[:, 1], marker='X', s=200)

for i,(center_x,center_y) in enumerate(centroids):
    plt.text(center_x, center_y, f'Clu_{i}', fontsize=10,color='red', ha='center', va='center')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('K-means Clustering in t-SNE Space')
#plt.legend(*sc.legend_elements(), title='clusters')
plt.savefig('V3_clustering_id.png',bbox_inches="tight",dpi=300)
plt.show()

In [ ]:
my_pos_df['class']=labels
plt.figure(figsize=(8, 6))
palette = sns.color_palette(cc.glasbey_light, n_colors=20)

ax=sns.scatterplot(x='X', y='Y', hue="class", 
                data=my_pos_df,palette=palette,s=20);

#plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='black', marker='X', s=150, label=centroid_labels)
for i,(center_x,center_y) in enumerate(centroids):
    plt.text(center_x, center_y, f'Clu_{i}', fontsize=10,color='black', ha='center', va='center')

In [ ]:
#Over-representation pathway analysis
#all the protein list, make it as the gene backgroud
gene_name_list=[]
for idx,i in enumerate(correlation_matrix.index):
    gene_name=i.split(".")[0]
    gene_name_list.append(gene_name)
print(gene_name_list)

#remove the duplicated from the gene backgroud
gene_name_list_res = []
[gene_name_list_res.append(x) for x in gene_name_list if x not in gene_name_list_res]
glist_backgroud=gene_name_list_res

In [ ]:
#iterate each class and apply pathway analysis
#select the top1 pathway to annotate the cluster
top_pathway=[]
for i in range(20):
    clu=my_pos_df[my_pos_df['class']==i]
    
    glist=[]
    for i in clu.index:
        glist.append(i.split(".")[0])
    
    enr = gp.enrichr(gene_list=glist, 
                 gene_sets=['WikiPathways_2019_Human'],
                 #gene_sets=['KEGG_2021_Human'],
                 background=glist_backgroud,
                 organism='human', 
                 outdir=None, 
                )
    top_pathway.append(enr.results.iloc[0,1])
    print(enr.results.iloc[0,1])

In [ ]:
#convert 2 lists to dictionary
#key would be index, value would be pathway
test_keys = range(20)
test_values = top_pathway

pathway_dic = {}
for key in test_keys:
    for value in test_values:
        pathway_dic[key] = value
        test_values.remove(value)
        break
pathway_dic

In [ ]:
my_pos_df['class_2'] = my_pos_df['class'].map(pathway_dic)
my_pos_df

In [ ]:
g=sns.scatterplot(x="X", y="Y", hue="class_2", 
                data=my_pos_df, palette=palette, s=25);
plt.legend(bbox_to_anchor=(1.02, 0.55), loc='upper left', borderaxespad=0)
g.set(yticklabels=[])
g.set(xticklabels=[])
g.set(ylabel=None)
g.set(xlabel=None)
g.tick_params(left=False)
g.tick_params(bottom=False)
sns.despine(left=True, bottom=True, right=True)
plt.savefig('V3_clustering_label.png',bbox_inches="tight",dpi=800)